In [35]:
from os import path,listdir
import csv
import re
from datetime import timedelta, date
import time

def getone(pattern,string):
    m=pattern.search(string)
    res=m.group(1) if m else ''
    return res.strip()

def parseminsec(string):
    string=string.replace('s','')
    m=0
    if 'm' in string:
        (m,s)=string.split('m')
        m=float(m.strip())
        s=float(s.strip())
    else:
        s=float(string.strip())
    return m*60+s

def clearhorsename(string):
    string=string.replace(')','').strip()
    ret=string
    if '(' in string:
        ret=string.split('(')[0]
    if ret=='':
        ret=string.split('(')[1]
    return ret.strip()

pCourse=re.compile(r'rp-raceTimeCourseName__name\s*">\s*([^<]+)')
pDate=re.compile(r'<span class="rp-raceTimeCourseName__date" data-test-selector="text-raceDate">([^<]+)')
pTitle=re.compile(r'<h2 class="rp-raceTimeCourseName__title">([^<]+)')
pClass=re.compile(r'<span class="rp-raceTimeCourseName_class">([^<]+)')
pBandAndAgesAllowed=re.compile(r'<span class="rp-raceTimeCourseName_ratingBandAndAgesAllowed">([^<]+)')
pDistance=re.compile(r'<span class="rp-raceTimeCourseName_distanceFull" data-test-selector="block-fullDistanceInd">([^<]+)')
pCondition=re.compile(r'<span class="rp-raceTimeCourseName_condition">([^<]+)')
pHurdles=re.compile(r'<span class="rp-raceTimeCourseName_hurdles">([^<]+)')
pPrizes=re.compile(r'<span class="rp-raceTimeCourseName__prizeMoneyTitle">[^<]+</span>([^<]+)')
pWinningTime=re.compile(r'Winning time:\s*<span class="rp-raceInfo__value">([^<]+)')
def parse_race(html):
    course=getone(pCourse,html)
    date=time.strftime('%Y-%m-%d',time.strptime(getone(pDate,html),'%d %b %Y'))
    title=getone(pTitle,html)
    rclass=getone(pClass,html).replace('(','').replace(')','')
    bandAndAgesAllowed=getone(pBandAndAgesAllowed,html).replace('(','').replace(')','').split(', ')
    if len(bandAndAgesAllowed)>1:
        band=bandAndAgesAllowed[0]
        ages=bandAndAgesAllowed[1]
    else:
        band=''
        ages=bandAndAgesAllowed[0]
    distance=getone(pDistance,html).replace('(','').replace(')','')
    condition=getone(pCondition,html)
    hurdles=getone(pHurdles,html)
    winningTime=getone(pWinningTime,html)
    slowby=''
    if '(slow by ' in winningTime:
        (winningTime,slowby)=winningTime.split('(slow by ')
        slowby=parseminsec(slowby.replace(')','').strip())
    winningTime=parseminsec(winningTime.strip())
    
    prizes=[]
    mm=pPrizes.findall(html)
    for m in mm:
        prizes.append(m.strip()[2:].replace(',',''))
    currency=''
    if 'ВЈ' in mm[0]:
        currency='GBP'
    print(course,date,title,rclass,band,ages,distance,condition,hurdles,prizes,currency,winningTime,slowby)
    parts=html.split('<tr class="rp-horseTable__mainRow" data-test-selector="table-row">')[1:]
    for part in parts:
        part=part.split('<tr class="rp-horseTable__hackRow">')[0].replace('вЂ“','')

        horseLink=getone(pHorseLink,part)
        horseName=getone(pHorseName,part)
        age=getone(pAge,part)
        saddle=getone(pSaddle,part).replace('.','')
        position=getone(pPosition,part)
        price=getone(pPrice,part)
        isFav='1' if 'F' in price else '0'
        price=price.replace('F','').replace('Evens','1/1')
        decimalPrice=''
        if price!='':
            (a,b)=price.split('/')
            decimalPrice=str(float(a)/float(b)+1).replace('.0','')
        trainerLink=getone(pTrainerLink,part)
        trainerLink='' if trainerLink=='0/' else trainerLink
        trainerName=getone(pTrainerName,part)
        jockeyLink=getone(pJockeyLink,part)
        jockeyLink='' if jockeyLink=='0/' else jockeyLink
        jockeyName=getone(pJockeyName,part)

        RPR=getone(pRPR,part)
        TR=getone(pTR,part)
        OR=getone(pOR,part)
        part=part.replace('<!--','').replace('-->','')
        weightSt=getone(pWeightSt,part)
        weightLb=getone(pWeightLb,part)
        overWeight=getone(pOverWeight,part)
        outHandicap=getone(pOutHandicap,part)
        headGear=getone(pHeadGear,part)
        part=part.replace('<span>','').replace('</span>','')
        positionL=getone(pPositionL,part).replace(']','').replace('ВЅ','.5').replace('Вј','.25').replace('Вѕ','.75')
        dist=''
        if '[' in positionL:
            (positionL,dist)=positionL.split('[')
            positionL=positionL.strip()
        if 'data-test-selector="block-pedigreeInfoFullResults">' in part:
            part=part.split('data-test-selector="block-pedigreeInfoFullResults">')[1]
            links=pHorseAncLink.findall(part)
            names=[clearhorsename(x) for x in pHorseAncName.findall(part)]
        else:
            links=names=['','','']
        comment=getone(pComment,part).split('(')[0].strip()
        print('horseName="{}",horseLink="{}",age="{}",saddle="{}",price="{}",decimalPrice="{}", isFav="{}",trainerLink="{}",trainerName="{}",jockeyLink="{}",jockeyName="{}",position="{}",positionL="{}",dist="{}",weightSt="{}",weightLb="{}",overWeight="{}",outHandicap="{}",headGear="{}",RPR="{}",TR="{}",OR="{}",names="{}",links="{}",comment="{}"'.format(horseName,horseLink,age,saddle,price,decimalPrice,isFav,trainerLink,trainerName,jockeyLink,jockeyName,position,positionL,dist,weightSt,weightLb,overWeight,outHandicap,headGear,RPR,TR,OR,names,links,comment))
            
pComment=re.compile(r'text-comments">\s*<td colspan="11">([^<]+)')
pHorseAncName=re.compile(r'<a href="/profile/horse/[^"]+" class="ui-profileLink ui-link ui-link_table js-popupLink">([^<]+)')
pHorseAncLink=re.compile(r'<a href="/profile/horse/([^"]+)" class="ui-profileLink ui-link ui-link_table js-popupLink">')
pRPR=re.compile(r'<td class="rp-horseTable__spanNarrow" data-ending="RPR" data-test-selector="full-result-rpr">([^<]+)')
pTR=re.compile(r'<td class="rp-horseTable__spanNarrow" data-ending="TS" data-test-selector="full-result-topspeed">([^<]+)')
pOR=re.compile(r'<td class="rp-horseTable__spanNarrow" data-ending="OR">([^<]+)')
pWeightSt=re.compile(r'<span class="rp-horseTable__st" data-ending="st" data-test-selector="horse-weight-st">([^<]+)')
pWeightLb=re.compile(r'<span data-ending="lb" data-test-selector="horse-weight-lb">([^<]+)')
pOverWeight=re.compile(r'<img src="https://www.rp-assets.com/pdfs/over_weight.gif" alt="Over-Weights" data-test-selector="img-overWeight">\s*<span>([^<]+)')
pOutHandicap=re.compile(r'<img src="https://www.rp-assets.com/pdfs/out_of_handicap.gif" alt="Out-of-Handicaps" data-test-selector="img-outOfHandicap">\s*<span>([^<]+)')
pHeadGear=re.compile(r'<span class="rp-horseTable__headGear">([^<]+)')
pAge=re.compile(r'<td class="rp-horseTable__spanNarrow rp-horseTable__spanNarrow_age" data-ending="yo" data-test-selector="horse-age">([^<]+)')
pTrainerLink=re.compile(r'<a href="/profile/trainer/([^"]+)"')
pTrainerName=re.compile(r'link-trainerName">([^<]+)')
pJockeyLink=re.compile(r'<a href="/profile/jockey/([^"]+)"')
pJockeyName=re.compile(r'link-jockeyName">([^<]+)')
pPrice=re.compile(r'<span class="rp-horseTable__horse__price">([^<]+)')
pHorseLink=re.compile(r'<a href="/profile/horse/([^"]+)"')
pHorseName=re.compile(r'link-horseName">([^<]+)')
pSaddle=re.compile(r'<span class="rp-horseTable__saddleClothNo">([^<]+)')
pPosition=re.compile(r'data-test-selector="text-horsePosition">([^<]+)')
pPositionL=re.compile(r'<span class="rp-horseTable__pos__length">([^<]+)')






In [42]:
with open('output/races/1.html') as f:
    html=f.read()
parse_race(html)


Longchamp (FR) 2008-10-26 Prix Mackensie Grieves (Handicap)   4yo+  Good To Soft  ['8456', '3382', '2537', '1691', '846'] GBP 81.0 1.0
horseName="Zaliana",horseLink="659144/zaliana",age="4",saddle="15",price="",decimalPrice="", isFav="0",trainerLink="13742/alex-fracas",trainerName="Alex Fracas",jockeyLink="2452/thierry-jarnet",jockeyName="Thierry Jarnet",position="1",positionL="&nbsp;",dist="",weightSt="8",weightLb="8",overWeight="",outHandicap="",headGear="",RPR="70",TR="67",OR="",names="['Daliapour', 'Gandelia', 'Ganges']",links="['479982/daliapour', '473620/gandelia', '63078/ganges']",comment=""
horseName="Auzi",horseLink="659379/auzi",age="5",saddle="1",price="",decimalPrice="", isFav="0",trainerLink="4819/j-rossi",trainerName="J Rossi",jockeyLink="79955/julien-auge",jockeyName="Julien Auge",position="2",positionL=".5",dist="",weightSt="9",weightLb="6",overWeight="",outHandicap="",headGear="",RPR="81",TR="77",OR="",names="['Commands', 'Time For Romance', 'Cure The Blues']",links="[

In [14]:
from selenium import webdriver
def load_races(rid):
    df=pd.read_csv('data/days.csv')
    driver = webdriver.Chrome(executable_path=r'../lib/chromedriver.exe')
    id=rid
    while id<len(df):
        link=df.iloc[id].link
        driver.get('https://www.racingpost.com/{}'.format(link))
        html = driver.page_source
        with open('output/races/{}.html'.format(id),'w+', encoding='utf8') as f:
            f.write(html)
        print('[{}]: {} done with {} bytes.'.format(id, link,len(html)))
        id+=1
        break
    driver.close()

In [16]:
rid=1
load_races(rid)

[1]: /results/211/longchamp/2008-10-26/469058 done with 813997 bytes.
